In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import GRU

In [18]:
# Load the dataset
df = pd.read_csv('..//dataset//final_dataset_compressed.csv')  # Replace with your dataset path

# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [19]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(320000, 1, 6) (320000,) (400000, 1, 6)


In [20]:
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
def build_optimized_rnn_bi_lstm():
    model = Sequential()

    # Deep RNN layers
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(SimpleRNN(units=100, return_sequences=True,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(SimpleRNN(units=100, return_sequences=True, kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Bidirectional LSTM layers
    model.add(Bidirectional(LSTM(units=100, return_sequences=True, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=100, return_sequences=True, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Bidirectional(LSTM(units=100, kernel_regularizer=l2(0.01))))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model with Adam optimizer (adjust learning rate)
    model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['mse', 'mae'])

    return model

In [21]:
optimized_model = build_optimized_rnn_bi_lstm()

# Train the model
optimized_history = optimized_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=1)

Epoch 1/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - loss: 2.6416 - mae: 0.0659 - mse: 0.0376 - val_loss: 0.0538 - val_mae: 0.0254 - val_mse: 0.0130
Epoch 2/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - loss: 0.0852 - mae: 0.0412 - mse: 0.0212 - val_loss: 0.0686 - val_mae: 0.0260 - val_mse: 0.0149
Epoch 3/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - loss: 0.0770 - mae: 0.0369 - mse: 0.0187 - val_loss: 0.0557 - val_mae: 0.0266 - val_mse: 0.0138
Epoch 4/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 29s 7ms/step - loss: 0.0752 - mae: 0.0361 - mse: 0.0183 - val_loss: 0.0658 - val_mae: 0.0263 - val_mse: 0.0156
Epoch 5/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 0.0734 - mae: 0.0350 - mse: 0.0177 - val_loss: 0.0620 - val_mae: 0.0297 - val_mse: 0.0158
Epoch 6/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 0.0740 - mae: 0.0359 - mse: 0.0183 - val_loss: 0.0822 - val_mae: 0.0296 - val_mse: 0.0191
Epoch 7/10
4000/4000 ━━━━━━━━━━━━━━━━━━━━ 28s 7ms/step - loss: 0.0709 - mae: 0.034

In [22]:
import numpy as np

# Set numpy to display full float numbers
np.set_printoptions(suppress=True, precision=10)

In [23]:
# # Evaluate the model
# loss, accuracy =optimized_model.evaluate(X_test, y_test, verbose=1)
# print(f'Test Accuracy: {accuracy:.2f}')

In [24]:

from sklearn.metrics import mean_squared_error
import numpy as np

# Make predictions
y_pred = optimized_model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step
Mean Squared Error (MSE): 0.0162941943854094
Root Mean Squared Error (RMSE): 0.1276487147816591


In [25]:
# #visualizing the loss and accuracy
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10,5))
# plt.subplot(1,2,1)
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')

# plt.subplot(1,2,2)
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train','validation'],loc='upper left')
# plt.show()
